In [1]:
import pymongo
import numpy as np
import pandas as pd 

In [2]:
try:
    #use your database name, user and password here:
    #mongodb://<dbuser>:<dbpassword>@<mlab_url>.mlab.com:57066/<database_name>
    with open("credentials.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
    conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
    print ("Connected successfully!!!")
    
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 
conn

Connected successfully!!!


MongoClient(host=['ds233895.mlab.com:33895'], document_class=dict, tz_aware=False, connect=True)

In [4]:
db=conn["agile_data_science_group_3"]

In [5]:
conn["agile_data_science_group_3"].collection_names()

['system.indexes', 'resource_1']

In [12]:
#db.resource_1.find_one()

#### Create users from generated name list

In [17]:
user_data = pd.read_csv('generated_names', header = None)


In [20]:
user_data.columns = ['user_name']

In [27]:
user_data['email'] = (user_data['user_name'].replace(' ','.')+'@email.com')


In [40]:
for i in range(0,50):
    user_data['email'][i]=user_data['email'][i].replace(' ','.')

In [42]:
user_data.head()

,user_name,email
0,Chauncey Steffen,Chauncey.Steffen@email.com
1,Kala Welter,Kala.Welter@email.com
2,Daron Kirtley,Daron.Kirtley@email.com
3,Wilfredo Jacobi,Wilfredo.Jacobi@email.com
4,Bess Fiala,Bess.Fiala@email.com


In [49]:
user_data['password'] = ''
for i in range(0,50):
    user_data['password'][i] = user_data['user_name'][i].split()[0]


In [50]:
user_data.head()

,user_name,email,password
0,Chauncey Steffen,Chauncey.Steffen@email.com,Chauncey
1,Kala Welter,Kala.Welter@email.com,Kala
2,Daron Kirtley,Daron.Kirtley@email.com,Daron
3,Wilfredo Jacobi,Wilfredo.Jacobi@email.com,Wilfredo
4,Bess Fiala,Bess.Fiala@email.com,Bess


In [51]:
### adding this data frame to mongo

In [57]:
#db.create_collection("users", { 'user_name': "string",
#                              'email': "string",
#                              'password': "string" } )

In [58]:
collection = db.users

In [59]:
collection.find_one()

In [62]:
#user_data.to_dict("records")

In [63]:
collection.insert_many(user_data.to_dict("records"))

In [65]:
collection.find_one()

{'_id': ObjectId('5a1aa2cb2cfaa80d6ff251f3'),
 'email': 'Chauncey.Steffen@email.com',
 'password': 'Chauncey',
 'user_name': 'Chauncey Steffen'}

#### Generate ratings

For each user I select number of recipes he will rate (1 to 5), then from recipes I select random recipes and add them random rating (1 to 5).

In [78]:
user_ids = collection.find({})


In [85]:
recipe_coll = db.resource_1

In [102]:
### select random recipe
import random

def get_random_recipe():
    count = recipe_coll.count()
    return recipe_coll.find()[random.randrange(count)]

In [97]:
rating_coll = db.ratings

In [123]:
for i in range(0,50):
    user = user_ids[i]["_id"]
    rec_count = random.randrange(1,5)
    for j in range(0,rec_count):
        recipe = get_random_recipe()
        #print(recipe["_id"])
        rate = random.randrange(0,5)
        rating_coll.insert_one({'user_id': user, 'recipe_id': recipe["_id"], 'rating':rate})

In [127]:
db.ratings.find_one()

{'_id': ObjectId('5a1aa9612cfaa80d6ff25315'),
 'rating': 1,
 'recipe_id': ObjectId('5a073c312cfaa84d14994d8e'),
 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4')}

In [132]:
for elem in db.ratings.find({}):
    print(elem)


{'_id': ObjectId('5a1aa9612cfaa80d6ff25315'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4'), 'recipe_id': ObjectId('5a073c312cfaa84d14994d8e'), 'rating': 1}
{'_id': ObjectId('5a1aa9612cfaa80d6ff25316'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f4'), 'recipe_id': ObjectId('5a073c312cfaa84d14994994'), 'rating': 1}
{'_id': ObjectId('5a1aa9632cfaa80d6ff2531f'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f7'), 'recipe_id': ObjectId('5a073c312cfaa84d14995dc5'), 'rating': 3}
{'_id': ObjectId('5a1aa9632cfaa80d6ff25320'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f8'), 'recipe_id': ObjectId('5a073c312cfaa84d14996b2f'), 'rating': 1}
{'_id': ObjectId('5a1aa9632cfaa80d6ff25321'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f8'), 'recipe_id': ObjectId('5a073c312cfaa84d14994986'), 'rating': 1}
{'_id': ObjectId('5a1aa9632cfaa80d6ff25322'), 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f8'), 'recipe_id': ObjectId('5a073c312cfaa84d1499541b'), 'rating': 4}
{'_id': ObjectId('5a1aa9602cfaa80d6ff25313'), 

In [135]:
db.ratings.find({}).count(True)

118